In [1]:
import folium

In [2]:
from pyproj import crs
import pandas as pd

from folium.plugins import HeatMap

from folium.plugins import MarkerCluster

from folium import IFrame, FeatureGroup, LayerControl

from folium.plugins import GroupedLayerControl

# Création de la carte interactive

In [21]:
Tree = pd.read_excel('data/20230717_Lacuzon Arbre_data.xlsx')

In [22]:
Tree = Tree.apply(lambda x: [f'{y:.0f}'for y in x] if x.name=='DEATH_YEAR' else x)


In [23]:
Tree[Tree["Génération"]==4]

,Unnamed: 0,ID,Sosa,Génération,Name,Genre,BIRTH_PLACE,BIRTH_DATE,MARRIAGE_PLACE,MARRIAGE_DATE,...,BIRTH_YEAR Réel ou Projeté,BIRTH_YEAR,DEATH_YEAR Réel ou Projeté,DEATH_YEAR,Age,Lieu normé Réel ou Projeté,Lieux_normé,Département,long,lat
9,7.0,@I1466526025@,8,4,Pierre Paul Emile Joseph PROST,M,"Dole, 39198, Jura, Franche-Comté, France",24 JUN 1896,NaN,NaN,...,R,1896,R,1982,86.0,R,"Gare de Dole, 42 Av. Aristide Briand, 39100 Do...",39.0,5.487996,47.096190
10,11.0,@I1487268917@,9,4,Yvonne Marie FRAUNIÉ,F,"Paris, Paris, Île-de-France, France",28 AUG 1900,NaN,NaN,...,R,1900,R,1990,90.0,R,"Paris, France",75.0,2.352222,48.856614
11,9.0,@I1475942410@,10,4,Louis GOUDAL,M,"Saint-Cyr-les-Champagnes, Dordogne, Nouvelle-A...",1 AUG 1909,NaN,NaN,...,R,1909,R,1988,79.0,R,"24270 Saint-Cyr-les-Champagnes, France",24.0,1.289729,45.378702
12,13.0,@I1475942459@,11,4,Odette LAJUGIE,F,NaN,1914,NaN,NaN,...,R,1914,R,2014,100.0,R,"19130 Vignols, France",19.0,1.389723,45.319120
13,8.0,@I0524@,12,4,Jean SALESSES,M,"Paris, Paris, Île-de-France, France",26 DEC 1904,NaN,NaN,...,R,1904,R,1983,79.0,R,"Paris, France",75.0,2.352222,48.856614
14,12.0,@I0525@,13,4,Mathilde Marie Charlotte RESCHIGNA,F,"Paris, Paris, Île-de-France, France",3 AUG 1908,NaN,NaN,...,R,1908,R,1997,89.0,R,"Paris, France",75.0,2.352222,48.856614
15,10.0,@I0520@,14,4,Armand Jean SCHWAAB,M,"Strasbourg, Alsace",9 JUL 1899,NaN,NaN,...,R,1899,R,1956,57.0,R,"Strasbourg, France",67.0,7.752111,48.573405
16,14.0,@I0521@,15,4,Lucie Charlotte KAPP,F,"Strasbourg, Alsace",13 OCT 1901,NaN,NaN,...,R,1901,R,1969,68.0,R,"Strasbourg, France",67.0,7.752111,48.573405


In [24]:
# Create a Map instance
m = folium.Map(location=[48.8639368381152, 2.349781581911858], zoom_start=6, control_scale=True)


In [25]:
# Définition de la foction pour afficher la popup
def HtmlPopup(location_info):
    html = """ \
    <h5>"""+location_info["Name"]+"""</h5><p style="line-height: 200%">"""
    
    html += str(location_info["BIRTH_YEAR"]) + """ (""" + str(location_info["BIRTH_YEAR Réel ou Projeté"])
    
    html += """) - """

    html += str(location_info["DEATH_YEAR"]) + """ (""" + str(location_info["DEATH_YEAR  Réel ou Projeté"])
    
    html += """)</p>"""
    
    return html

In [26]:
GenerationList = Tree["Génération"].drop_duplicates().sort_values().values

GenerationLayer = []

for i in range(len(GenerationList)):
    Gene = GenerationList[i]
    globals()[f"marker_cluster_G{Gene}"] = MarkerCluster(name=f"G{Gene}", overlay=True).add_to(m)
   
    GenerationLayer.append(globals()[f"marker_cluster_G{Gene}"])
    for index, location_info in Tree[Tree["Génération"]==Gene].iterrows():
        location = [location_info["lat"], location_info["long"]]
        
        popup = HtmlPopup(location_info)
        
        folium.Marker(
            location=location,
            popup=popup,
        ).add_to(globals()[f"marker_cluster_G{Gene}"])
    
LayerControl().add_to(m)

GroupedLayerControl(
    groups={'Génération': GenerationLayer},
    exclusive_groups=True,
    collapsed=False,
).add_to(m)


In [27]:
m

In [28]:
outfp = "docs/index.html"
m.save(outfp)

# Parser du Gedcom

***Objectif*** : Travailler un gedcom afin de créer un dataframe panda exploitable

In [82]:
from ged4py import GedcomReader
from ged4py.date import DateValueVisitor

import pandas as pd


In [7]:
# Path to your `.ged` file
path = 'data/20230713_Prost Lacuzon-Export_all.ged'

# Initialize the parser
parser = GedcomReader(path, encoding="utf-8")


In [198]:
# Fonction ajout individu

def ParseIndividuGeneration(ID, Sosa, Genera):
    for i, indi in enumerate(parser.records0("INDI")):
        if indi.xref_id == ID:

            IndividuGénération['ID'].append(indi.xref_id)
            IndividuGénération['Sosa'].append(Sosa)
            IndividuGénération['Génération'].append(Genera)
            IndividuGénération['Name'].append(indi.name.format())
            IndividuGénération['Genre'].append(indi.sub_tag_value("SEX"))
            IndividuGénération['BIRTH_PLACE'].append(indi.sub_tag_value("BIRT/PLAC"))
            IndividuGénération['BIRTH_DATE'].append(indi.sub_tag_value("BIRT/DATE"))
            IndividuGénération['MARRIAGE_PLACE'].append(indi.sub_tag_value("MARR/PLAC"))
            IndividuGénération['MARRIAGE_DATE'].append(indi.sub_tag_value("MARR/DATE"))
            IndividuGénération['DEATH_PLACE'].append(indi.sub_tag_value("DEAT/PLAC"))
            IndividuGénération['DEATH_DATE'].append(indi.sub_tag_value("DEAT/DATE"))        


            if indi.father != None:
                IndividuGénération['Father'].append(indi.father.xref_id) 
            else: IndividuGénération['Father'].append(None)

            if indi.mother != None:
                IndividuGénération['Mother'].append(indi.mother.xref_id)
            else: IndividuGénération['Mother'].append(None)
            break

In [251]:
# Dictionnaire des individus par génération
IndividuGénération = {'ID':[], 'Sosa':[], "Génération" :[] ,"Name" :[], "Genre" :[], "BIRTH_PLACE" :[], "BIRTH_DATE" :[], "MARRIAGE_PLACE" :[], "MARRIAGE_DATE" :[], "DEATH_PLACE" :[], "DEATH_DATE" :[], "Father" :[], "Mother" :[]}



In [252]:
# Initialisation du sujet 0

ID = '@I0024@'
Sosa = 1
Genera = 1
            
ParseIndividuGeneration(ID, 1, 1)

In [253]:

DfIndividuGénération = pd.DataFrame.from_dict(IndividuGénération)

In [254]:
#Boucle itération génération
Genera = 1
Gen = True

while Gen == True:
    Gen = False
    
    #Rajout père
    for index, DfIndividu_info in DfIndividuGénération[DfIndividuGénération['Génération']==Genera].iterrows():
        if DfIndividu_info['Father'] != None:
            ID = DfIndividu_info['Father']
            Sosa = DfIndividu_info['Sosa'] *2
            GeneraInd = Genera + 1

            ParseIndividuGeneration(ID, Sosa, GeneraInd)
            Gen = True

    #Rajout mère
    for index, DfIndividu_info in DfIndividuGénération[DfIndividuGénération['Génération']==Genera].iterrows():
        if DfIndividu_info['Mother'] != None:
            ID = DfIndividu_info['Mother']
            Sosa = DfIndividu_info['Sosa'] *2 + 1
            GeneraInd = Genera + 1

            ParseIndividuGeneration(ID, Sosa, GeneraInd)
            Gen = True
    
    DfIndividuGénération = pd.DataFrame.from_dict(IndividuGénération)
    Genera += 1

In [258]:
#Suppression des doublons
DfIndividuGénération= DfIndividuGénération.drop_duplicates(subset=['ID'])

In [259]:
DfIndividuGénération.to_excel("Lacuzon Arbre.xlsx")

# Geolocalisation

In [310]:
from geopy.geocoders import GoogleV3

import requests, json

In [311]:

# Opening JSON file
f = open("Param/param.json")

# returns JSON object as 
# a dictionary
param = json.load(f)


In [312]:
# Création Geocoder Google
locator = GoogleV3(api_key=param['GooglePlateform_API']['api_key'])


In [316]:
dFLieu = pd.read_excel('Data/Lieu Genealogie.xlsx')

In [344]:
Lieu_Info = {'Lieu':[], 'location':[], 'long':[], 'lat':[]}

for index, DfLieu_info in dFLieu.iterrows():
    Lieu_Info['Lieu'].append(DfLieu_info.values[0])
    location = locator.geocode(DfLieu_info.values[0])
    
    if location == None:
        Lieu_Info['location'].append(None)
        Lieu_Info['long'].append(None)
        Lieu_Info['lat'].append(None)      
    else:
        Lieu_Info['location'].append(location)
        Lieu_Info['long'].append(location.longitude)
        Lieu_Info['lat'].append(location.latitude)

In [347]:
DfLieu_Info = pd.DataFrame.from_dict(Lieu_Info)

In [349]:
DfLieu_Info.to_excel("LieuGeoCode.xlsx")